https://wikidocs.net/24949 참고 링크

# 잠재 의미 분석(Latent Semantic Analysis , LSA) 
- LSA 를 수행하기 위해서는 선형대수학의 특이값분해(SVD)의 개념에 대한 이해가 필요

### 1. 특이값 분해(Singular Value Decomposition, SVD)

- SVD란 , A가 m x n 행렬일때 , 다음과 같이 3개의 행렬곱으로 분해하는 것
$$A=UΣV^T$$

- 여기서 각 3개의 행렬은 다음과 같은 조건을 만족합니다.
$$U:m×m 직교행렬 (AA^T=U(ΣΣT)U^T)$$
$$V:n×n 직교행렬 (A^TA=V(ΣTΣ)V^T)$$
$$Σ:m×n 직사각 대각행렬$$


### 2. 절단된 SVD (Truncated SVD)
- 위 1번에서는 full SVD라고 불린다.
- 하지만 LSA 에서는 일부 벡터들을 삭제시킨 절단된 SVD(truncated SVD)사용
- 아래 그림으로 , 확실히 벡터 차원의 수가 작아짐을 볼 수 있다.

![](https://wikidocs.net/images/page/24949/svd%EC%99%80truncatedsvd.PNG)
- 위 과정을 통해 , 상대적으로 중요하지 않은 정보를 삭제해줌
- 계산 비용 또한 낮아진다

### 3. 잠재 의미 분석 (Latent Semantic Analysis , LSA)
-DTM 공부에 활용했던 예제를 다시 사용

문서1 : 먹고 싶은 사과<br>
문서2 : 먹고 싶은 바나나<br>
문서3 : 길고 노란 바나나 바나나<br>
문서4 : 저는 과일이 좋아요<br>
<center> DTM </center>

과일이| 길고 |노란  |먹고|바나나|사과|싶은|저는|좋아요
---   |---   |---   |--- |---   |--- |--- |--- |--- 
문서1|	0   |0	   |0	|1	|0	|1	|1	|0	|0
문서2|	0   |0	   |0	|1	|1	|0	|1	|0	|0
문서3|	0	|1	   |1	|0	|2	|0	|0	|0	|0
문서4|	1	|0	   |0	|0	|0	|0	|0	|1	|1

- 위를 바탕으로 실습

# Full SVD 실습

In [1]:
import numpy as np
A=np.array([[0,0,0,1,0,1,1,0,0],[0,0,0,1,1,0,1,0,0],[0,1,1,0,2,0,0,0,0],[1,0,0,0,0,0,0,1,1]])
np.shape(A)

(4, 9)

In [2]:
U, s, VT = np.linalg.svd(A, full_matrices = True)

In [3]:
print(U.round(2))  #소수점 너무 길이서 두번째까지만 출력
np.shape(U)

[[-0.24  0.75  0.   -0.62]
 [-0.51  0.44 -0.    0.74]
 [-0.83 -0.49 -0.   -0.27]
 [-0.   -0.    1.    0.  ]]


(4, 4)

In [4]:
print(s.round(2))
np.shape(s)

[2.69 2.05 1.73 0.77]


(4,)

- 여기서 s는 대각행렬이 아닌 , 특이값의 리스트로 변환해야함

In [5]:
S = np.zeros((4, 9)) # 대각 행렬의 크기인 4 x 9의 임의의 행렬 생성
S[:4, :4] = np.diag(s) # 특이값을 대각행렬에 삽입
print(S.round(2))
np.shape(S)

[[2.69 0.   0.   0.   0.   0.   0.   0.   0.  ]
 [0.   2.05 0.   0.   0.   0.   0.   0.   0.  ]
 [0.   0.   1.73 0.   0.   0.   0.   0.   0.  ]
 [0.   0.   0.   0.77 0.   0.   0.   0.   0.  ]]


(4, 9)

- 4 × 9의 크기를 가지는 대각 행렬 S가 생성되었습니다.
- 2.69 > 2.05 > 1.73 > 0.77 순으로 값이 내림차순을 보이는 것을 확인할 수 있습니다.

In [6]:
print(VT.round(2))
np.shape(VT)

[[-0.   -0.31 -0.31 -0.28 -0.8  -0.09 -0.28 -0.   -0.  ]
 [ 0.   -0.24 -0.24  0.58 -0.26  0.37  0.58 -0.   -0.  ]
 [ 0.58 -0.    0.    0.   -0.    0.   -0.    0.58  0.58]
 [ 0.   -0.35 -0.35  0.16  0.25 -0.8   0.16 -0.   -0.  ]
 [-0.   -0.78 -0.01 -0.2   0.4   0.4  -0.2   0.    0.  ]
 [-0.29  0.31 -0.78 -0.24  0.23  0.23  0.01  0.14  0.14]
 [-0.29 -0.1   0.26 -0.59 -0.08 -0.08  0.66  0.14  0.14]
 [-0.5  -0.06  0.15  0.24 -0.05 -0.05 -0.19  0.75 -0.25]
 [-0.5  -0.06  0.15  0.24 -0.05 -0.05 -0.19 -0.25  0.75]]


(9, 9)

- 9 × 9의 크기를 가지는 직교 행렬 VT(V의 전치 행렬)가 생성되었습니다. 
- 즉, U × S × VT를 하면 기존의 행렬 A가 나와야 합니다.
- Numpy의 allclose()는 2개의 행렬이 동일하면 True를 리턴합니다. 
- 이를 사용하여 정말로 기존의 행렬 A와 동일한지 확인해보겠습니다.

In [7]:
np.allclose(A, np.dot(np.dot(U,S), VT).round(2))


True

# Truncated SVD 실습
- truncated SVD 는 t를 정하는것이 중요함
- 여기서 t = 2 로 두기로 함 .
- 행렬 S 내의 특이값 중에서 상위2개만 남기고 나머지 제거

In [8]:
S=S[:2,:2]
print(S.round(2))

[[2.69 0.  ]
 [0.   2.05]]


In [9]:
U=U[:,:2]
print(U.round(2))

[[-0.24  0.75]
 [-0.51  0.44]
 [-0.83 -0.49]
 [-0.   -0.  ]]


- 2개의 열만 남기고 모두 제거 
- 이제 행렬 V의 전치 행렬인 VT에 대해서 2개 행만 남기고 제거

In [10]:
VT=VT[:2,:]
print(VT.round(2))

[[-0.   -0.31 -0.31 -0.28 -0.8  -0.09 -0.28 -0.   -0.  ]
 [ 0.   -0.24 -0.24  0.58 -0.26  0.37  0.58 -0.   -0.  ]]


- 이제 축소된 행렬 U, S, VT에 대해서 다시 U × S × VT연산을 하면 기존의 A와는 다른 결과가 나오게 됩니다. 
- 값이 손실되었기 때문에 이 세 개의 행렬로는 이제 기존의 A행렬을 복구할 수 없습니다.
- U × S × VT연산을 해서 나오는 값을 A_prime이라 하고 기존의 행렬 A와 값을 비교해보도록 하겠습니다.

In [11]:
A_prime=np.dot(np.dot(U,S), VT)
print(A,'\n')
print(A_prime.round(2))

[[0 0 0 1 0 1 1 0 0]
 [0 0 0 1 1 0 1 0 0]
 [0 1 1 0 2 0 0 0 0]
 [1 0 0 0 0 0 0 1 1]] 

[[ 0.   -0.17 -0.17  1.08  0.12  0.62  1.08 -0.   -0.  ]
 [ 0.    0.2   0.2   0.91  0.86  0.45  0.91  0.    0.  ]
 [ 0.    0.93  0.93  0.03  2.05 -0.17  0.03  0.    0.  ]
 [ 0.    0.    0.    0.    0.   -0.    0.    0.    0.  ]]


>- 차원을 많이 축소시켰는데도 , 대체적으로 비슷한 경향을 띄는것을 볼 수 있다.
>- 기존에 0인 값들은 0에 가까운 값이 나오고 , 1인 값들은 1에 나오는 것을볼 수 있다.
>- 하지만 , 예외적으로 문제가 있는 부분은 존재한다 .

# 뉴스데이터를 통한 실습
- 여기서는 LSA를 사용해서 문서의 수를 원하는 토픽의 수로 압축한 뒤에 각 토픽당 가장 중요한 단어 5개를 출력하는 실습으로 토픽 모델링을 수행해보도록 하겠습니다.

### 1) 뉴스 데이터에 대한 이해 

In [12]:
import pandas as pd
from sklearn.datasets import fetch_20newsgroups
dataset = fetch_20newsgroups(shuffle = True , random_state=1, remove=('headers','footers','quotes'))
documents = dataset.data
len(documents)

11314

훈련에 사용할 뉴스는 총 11,314개이고 , 첫번재 훈련용 뉴스 출력해서 확인해보자

In [13]:
documents[1]

"\n\n\n\n\n\n\nYeah, do you expect people to read the FAQ, etc. and actually accept hard\natheism?  No, you need a little leap of faith, Jimmy.  Your logic runs out\nof steam!\n\n\n\n\n\n\n\nJim,\n\nSorry I can't pity you, Jim.  And I'm sorry that you have these feelings of\ndenial about the faith you need to get by.  Oh well, just pretend that it will\nall end happily ever after anyway.  Maybe if you start a new newsgroup,\nalt.atheist.hard, you won't be bummin' so much?\n\n\n\n\n\n\nBye-Bye, Big Jim.  Don't forget your Flintstone's Chewables!  :) \n--\nBake Timmons, III"

사용할 사이킷런에서 제공되는 뉴스데이터는 뉴스별 target_name에 20개의 카테고리가 저장되어있음

In [14]:
print(dataset.target_names)

['alt.atheism', 'comp.graphics', 'comp.os.ms-windows.misc', 'comp.sys.ibm.pc.hardware', 'comp.sys.mac.hardware', 'comp.windows.x', 'misc.forsale', 'rec.autos', 'rec.motorcycles', 'rec.sport.baseball', 'rec.sport.hockey', 'sci.crypt', 'sci.electronics', 'sci.med', 'sci.space', 'soc.religion.christian', 'talk.politics.guns', 'talk.politics.mideast', 'talk.politics.misc', 'talk.religion.misc']


### 2) 텍스트 전처리
- 정제과정을 진행해야함
- 정규표현식을 통한 구두점 , 숫자 , 특수 문자 제거

In [15]:
news_df = pd.DataFrame({'document' : documents})
#특수 문자 제거
news_df['clean_doc'] = news_df['document'].str.replace("[^a-zA-Z]", " ")
#길이가 3이하인 단어는 제거 (길이가 짧은 단어 제거)
news_df['clean_doc'] = news_df['clean_doc'].apply(lambda x: ' '.join([w for w in x.split() if len(w)>3]))
# 전체 단어에 대한 소문자 변환
news_df['clean_doc'] = news_df['clean_doc'].apply(lambda x: x.lower())

In [16]:
news_df['clean_doc'][0]  # 잘 정제되었음을 확인 할 수 있다

'well sure about story seem biased what disagree with your statement that media ruin israels reputation that rediculous media most israeli media world having lived europe realize that incidences such described letter have occured media whole seem ignore them subsidizing israels existance europeans least same degree think that might reason they report more clearly atrocities what shame that austria daily reports inhuman acts commited israeli soldiers blessing received from government makes some holocaust guilt away after look jews treating other races when they power unfortunate'

- 토큰화를 해준 후에 불용어를 제거

In [17]:
from nltk.corpus import stopwords
stop_words = stopwords.words('english') # NLTK로부터 불용어를 받아옵니다.
tokenized_doc = news_df['clean_doc'].apply(lambda x: x.split()) # 토큰화
tokenized_doc = tokenized_doc.apply(lambda x: [item for item in x if item not in stop_words])
#불용어 제거

news_df['clean_doc'] = tokenized_doc.apply(lambda x : " ".join(x))
#불용어까지 제거한 후에 , 그 값을 데이터프레임에 넣어주기

In [18]:
print(news_df['clean_doc'][0])
# 잘되었나 확인

well sure story seem biased disagree statement media ruin israels reputation rediculous media israeli media world lived europe realize incidences described letter occured media whole seem ignore subsidizing israels existance europeans least degree think might reason report clearly atrocities shame austria daily reports inhuman acts commited israeli soldiers blessing received government makes holocaust guilt away look jews treating races power unfortunate


### 3) TF-IDF 행렬 만들기 
- 정상적으로 불용어까지 제거되었음을 확인하고 수행

In [22]:
news_df['clean_doc'][1]  # 불용어 존재하지 않음을 볼 수 있다.

'yeah expect people read actually accept hard atheism need little leap faith jimmy logic runs steam sorry pity sorry feelings denial faith need well pretend happily ever anyway maybe start newsgroup atheist hard bummin much forget flintstone chewables bake timmons'

- TfidfVectorizer 를 통해 단어 1,000개에 대한 TF-IDF 행렬을 만들것
- 여기서는 상위 1,000개의 단어로 제한하도록 함

In [23]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer(stop_words='english', 
max_features= 1000, # 상위 1,000개의 단어를 보존 
max_df = 0.5, 
smooth_idf=True)

X = vectorizer.fit_transform(news_df['clean_doc'])
X.shape # TF-IDF 행렬의 크기 확인

(11314, 1000)

### 4) 토픽 모델링(Topic Modeling)

- 절단된SVD (truncated SVD)를 사용해 20개의 차원으로 차원축소 할것.
- 즉 , 20개의 토픽을 가졌다고 가정한 후에 토픽 모델링을 시도.
- 토픽의 숫자는 n_components의 파라미터로 지정이 가능

In [24]:
from sklearn.decomposition import TruncatedSVD
svd_model = TruncatedSVD(n_components=20, algorithm='randomized',n_iter=100, random_state=122)
svd_model.fit(X)
len(svd_model.components_)

20

여기서 svd_model.componets_는 앞서 배운 LSA에서 VT에 해당됩니다.

In [25]:
np.shape(svd_model.components_)

(20, 1000)

정확하게 토픽의 수 t x 단어의 수의 크기를 가지는 것을 볼 수 있다.

In [28]:
terms = vectorizer.get_feature_names()  #단어의 집합이며 , 1,000개의 단어가 저장되있다
terms

['ability',
 'able',
 'accept',
 'access',
 'according',
 'account',
 'action',
 'actions',
 'actual',
 'actually',
 'added',
 'addition',
 'additional',
 'address',
 'administration',
 'advance',
 'advice',
 'agencies',
 'agree',
 'algorithm',
 'allow',
 'allowed',
 'allows',
 'amendment',
 'america',
 'american',
 'americans',
 'analysis',
 'angeles',
 'anonymous',
 'answer',
 'answers',
 'anti',
 'anybody',
 'apparently',
 'appear',
 'appears',
 'apple',
 'application',
 'applications',
 'apply',
 'appreciate',
 'appreciated',
 'approach',
 'appropriate',
 'april',
 'arab',
 'archive',
 'area',
 'areas',
 'argument',
 'arguments',
 'armenia',
 'armenian',
 'armenians',
 'arms',
 'army',
 'article',
 'articles',
 'asked',
 'asking',
 'assume',
 'assuming',
 'atheism',
 'atheists',
 'attack',
 'attempt',
 'author',
 'authority',
 'available',
 'average',
 'avoid',
 'away',
 'background',
 'base',
 'baseball',
 'based',
 'basic',
 'basically',
 'basis',
 'begin',
 'beginning',
 'belief

In [29]:
def get_topics(components, feature_names, n=5):
    for idx, topic in enumerate(components):
        print("Topic %d:" % (idx+1), [(feature_names[i], topic[i].round(5)) for i in topic.argsort()[:-n - 1:-1]])
get_topics(svd_model.components_,terms)

Topic 1: [('like', 0.21386), ('know', 0.20046), ('people', 0.19293), ('think', 0.17805), ('good', 0.15128)]
Topic 2: [('thanks', 0.32888), ('windows', 0.29088), ('card', 0.18069), ('drive', 0.17455), ('mail', 0.15111)]
Topic 3: [('game', 0.37064), ('team', 0.32443), ('year', 0.28154), ('games', 0.2537), ('season', 0.18419)]
Topic 4: [('drive', 0.53324), ('scsi', 0.20165), ('hard', 0.15628), ('disk', 0.15578), ('card', 0.13994)]
Topic 5: [('windows', 0.40399), ('file', 0.25436), ('window', 0.18044), ('files', 0.16078), ('program', 0.13894)]
Topic 6: [('chip', 0.16114), ('government', 0.16009), ('mail', 0.15625), ('space', 0.1507), ('information', 0.13562)]
Topic 7: [('like', 0.67086), ('bike', 0.14236), ('chip', 0.11169), ('know', 0.11139), ('sounds', 0.10371)]
Topic 8: [('card', 0.46633), ('video', 0.22137), ('sale', 0.21266), ('monitor', 0.15463), ('offer', 0.14643)]
Topic 9: [('know', 0.46047), ('card', 0.33605), ('chip', 0.17558), ('government', 0.1522), ('video', 0.14356)]
Topic 10

# 5. LSA의 장단점(Pros and Cons of LSA)

- 장점
    - LSA는 쉽고 빠르게 구현 가능
    - 단어의 잠재적인 의미를 이끌어 유사도 계산에 좋은 성능 이끔
- 단점
    - 이미 계산된 LSA에 새로운 데이터 추가하여 계산하려고하면 처음부터 다시계산해야함
    - 새로운 정보에 대해 업데이트 어려움

이러한 단점으로 인해 LSA 대신에 Word2Vec 등 단어의 의미를 벡터화 할 수 있는 또다른 방법론인 인공 신경망 기반의 방법론이 각광받고 있다.